# Bio-JEPA AC Explainer


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import copy
import math
import numpy as np
import glob
import os
import json
from pathlib import Path
import matplotlib.pyplot as plt
import random
from dataclasses import dataclass

In [2]:
torch.manual_seed(1337)
random.seed(1337)

In [3]:
data_dir = Path('/Users/djemec/data/jepa')
shard_dir = data_dir / 'tokenized'
metadata_path = data_dir / 'perturbation_map.json'
checkpoint_dir = data_dir / 'checkpoint'

## Text Prep/Tokenization

We previously loaded and saved our data into a tokenized files.  For a full explainer, look at the data_prep_explainer.ipynb notebook.   In this step we'll pull the data from the files to use for training.  First we'll start by getting all the files in the tokenized directory. 

In [4]:
data_files = sorted(shard_dir.glob('*.npz'))
print(f'Found {len(data_files)} shards.')

Found 30 shards.


Now that we have the files, we need just a single file. Because of how we tokanized our files, we do not have to load them in any particular order so we'll select a random file from the list.  In training, we'd want to remove this from the file list to ensure we go through the full epoch before restarting.   

In [5]:
random_file_idx = np.random.randint(len(data_files))
file_path = data_files[random_file_idx]
random_file_idx, file_path

(19, PosixPath('/Users/djemec/data/jepa/tokenized/shard_0019.npz'))

# Modeling

## Data Loading


To start, we need to get enough data to run the forward and backward passes.  Since our total dataset is likely too big to be held in memory all at once in real practice, we will read just enough file information into memory so that we can run the passes, leaving memory and compute to be used on the passes instead of static data holding. 

Recall that we saved the data in the following structure:

```
{
    'control':[array of network expression information],
    'case':[array of network expression information],
    'action_ids':[array of ids of the perturbation]
}
```

Because of this, taking the same index out of each array gives us our action_id, pertrubed data, and control data. For training we'll do more than 1 cell at a time.  We'll use a variable `BATCH_SIZE` to control how much data to train on at a time.  In this case we'll start with 2. During training we'll want a increase our batch size.  


In [6]:
BATCH_SIZE = 2
with np.load(file_path) as data:
    # Keys: 'control', 'treated', 'action_ids'
    n_rows = data['action_ids'].shape[0]
    row_idx = np.random.randint(n_rows, size=BATCH_SIZE)
    
    control_raw = data['control'][row_idx]
    case_raw = data['case'][row_idx]
    act_id = data['action_ids'][row_idx]

x_control = torch.tensor(control_raw)
x_case = torch.tensor(case_raw)
action_id = torch.tensor(act_id, dtype=torch.long)

row_idx, action_id, x_control, x_case

(array([6448, 9579]),
 tensor([1499,  777]),
 tensor([[0.6825, 0.5222, 0.5322,  ..., 0.3849, 0.2634, 0.5255],
         [0.7225, 0.5116, 0.6232,  ..., 0.4091, 0.3447, 0.5609]]),
 tensor([[0.7002, 0.6049, 0.5186,  ..., 0.3651, 0.1818, 0.5686],
         [0.7559, 0.6287, 0.6083,  ..., 0.4122, 0.3186, 0.5887]]))

## Forward pass

**RoPE**

In [7]:
class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_seq_len=2048):
        super().__init__()
        # Ensure dim is the head dimension, not the full embedding dimension
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        t = torch.arange(max_seq_len).type_as(inv_freq)
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer('emb', emb)

    def forward(self, x):
        # x: [Batch, Seq, Dim]
        # We only care about the sequence length here
        n = x.shape[1]
        # Returns [Seq, Head_Dim]
        return self.emb[:n, :].cos(), self.emb[:n, :].sin()

def apply_rotary_pos_emb(q, k, cos, sin):
    # q, k: [Batch, Heads, Seq, Head_Dim]
    # cos, sin: [Seq, Head_Dim] -> reshape to [1, 1, Seq, Head_Dim]
    
    cos = cos.unsqueeze(0).unsqueeze(0)
    sin = sin.unsqueeze(0).unsqueeze(0)

    # Standard RoPE rotation logic
    # split last dim into half
    q_d = q.shape[-1] // 2
    k_d = k.shape[-1] // 2
    
    q1, q2 = q[..., :q_d], q[..., q_d:]
    k1, k2 = k[..., :k_d], k[..., k_d:]
    
    q_rotated = torch.cat((-q2, q1), dim=-1)
    k_rotated = torch.cat((-k2, k1), dim=-1)
    
    q_out = (q * cos) + (q_rotated * sin)
    k_out = (k * cos) + (k_rotated * sin)
    
    return q_out, k_out

**Multi-Headed Attention**

In [8]:
class BioMultiHeadAttention(nn.Module):
    # mirrors nn.MultiheadAttention(dim, heads, batch_first=True) 
    def __init__(self, config):
        super().__init__()
        assert config.embed_dim % config.heads == 0
        
        self.head_dim = config.embed_dim // config.heads
        self.heads = config.heads
        self.embed_dim = config.embed_dim
        
        # Projections
        self.q_proj = nn.Linear(config.embed_dim, config.embed_dim)
        self.k_proj = nn.Linear(config.embed_dim, config.embed_dim)
        self.v_proj = nn.Linear(config.embed_dim, config.embed_dim)
        
        self.c_proj = nn.Linear(config.embed_dim, config.embed_dim)

    def forward(self, x, cos, sin):
        B, T, C = x.size() # Batch, Seq, Embed Dim
        
        # 1. Project
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)

        # 2. Reshape for multi-head attention
        # (B, T, nh, hs) -> (B, nh, T, hs)
        q = q.view(B, T, self.heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.heads, self.head_dim).transpose(1, 2)

        # 3. Apply RoPE to Q and K (Rotary is applied per head)
        q, k = apply_rotary_pos_emb(q, k, cos, sin)

        # 4. Attention
        # is_causal=False because this is a bidirectional encoder
        y = F.scaled_dot_product_attention(q, k, v, is_causal=False)
        
        # 5. Reassemble
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        
        return y

**MLP**

In [9]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.embed_dim, int(config.mlp_ratio * config.embed_dim))
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(int(config.mlp_ratio * config.embed_dim), config.embed_dim)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

**Hidden Transformer Block**

In [10]:
class PathwayBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.embed_dim)
        self.attn = BioMultiHeadAttention(config)
        self.ln_2 = nn.LayerNorm(config.embed_dim)
        self.mlp = MLP(config)

    def forward(self, x, cos, sin):
        # 1. Attention with RoPE
        x_norm = self.ln_1(x)
        # Pass cos/sin into attn to be applied to Q/K
        attn_out = self.attn(x_norm, cos, sin)
        x = x + attn_out

        # 2. MLP
        x = x + self.mlp(self.ln_2(x))
        return x

**Main Pathway Model**

In [11]:
@dataclass
class PathwayEncoderConfig:
    num_pathways: int = 1024 
    n_layer: int = 24 
    heads: int = 12
    embed_dim: int = 768
    mlp_ratio: float = 4.0 # Changed to float for precision

class PathwayEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            # input projects to embedding
            input_proj = nn.Linear(1, config.embed_dim),
            
            # RoPE needs the HEAD dimension, not the full embedding dimension
            rope = RotaryEmbedding(config.embed_dim // config.heads),
            
            # transformer block
            blocks = nn.ModuleList([PathwayBlock(config) for _ in range(config.n_layer)]),
            
            # final layer norm
            ln_f = nn.LayerNorm(config.embed_dim) 
        ))

        self.apply(self._init_weights)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
        
    def forward(self, x):
        # x: [Batch, Num_Pathways]
        x = x.unsqueeze(-1) # [B, N, 1]
        x = self.transformer.input_proj(x) # [B, N, Dim]
        
        # Generate RoPE cache
        # cos, sin are [Seq, Head_Dim]
        cos, sin = self.transformer.rope(x)
        cos, sin = cos.to(x.device), sin.to(x.device)
        
        for block in self.transformer.blocks:
            x = block(x, cos, sin)
            
        x = self.transformer.ln_f(x)

        return x

### Create The Student And Teacher Models

In [12]:
n_embd = 4
n_heads = 2
n_pathways = 1024
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
student = PathwayEncoder(PathwayEncoderConfig(n_layer= 1, heads=n_heads, embed_dim= n_embd))
student

PathwayEncoder(
  (transformer): ModuleDict(
    (input_proj): Linear(in_features=1, out_features=4, bias=True)
    (rope): RotaryEmbedding()
    (blocks): ModuleList(
      (0): PathwayBlock(
        (ln_1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        (attn): BioMultiHeadAttention(
          (q_proj): Linear(in_features=4, out_features=4, bias=True)
          (k_proj): Linear(in_features=4, out_features=4, bias=True)
          (v_proj): Linear(in_features=4, out_features=4, bias=True)
          (c_proj): Linear(in_features=4, out_features=4, bias=True)
        )
        (ln_2): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=4, out_features=16, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=16, out_features=4, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
  )
)

**Teacher** Now let's make a copy for the teacher. Since the teacher does not require updates, we can update the model to not require gradient updates. 

In [14]:
teacher = copy.deepcopy(student)
teacher

PathwayEncoder(
  (transformer): ModuleDict(
    (input_proj): Linear(in_features=1, out_features=4, bias=True)
    (rope): RotaryEmbedding()
    (blocks): ModuleList(
      (0): PathwayBlock(
        (ln_1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        (attn): BioMultiHeadAttention(
          (q_proj): Linear(in_features=4, out_features=4, bias=True)
          (k_proj): Linear(in_features=4, out_features=4, bias=True)
          (v_proj): Linear(in_features=4, out_features=4, bias=True)
          (c_proj): Linear(in_features=4, out_features=4, bias=True)
        )
        (ln_2): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=4, out_features=16, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=16, out_features=4, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
  )
)

In [15]:
for p in teacher.parameters():
    p.requires_grad = False

### Forward Pass: Teacher
Teacher encodes the Target (Case). We'll make sure to run it without gradient accumulation.  This teacher creates an embedding of the case cell

In [16]:
with torch.no_grad():
    target_latents = teacher(x_case)
target_latents

tensor([[[ 0.7197,  0.8458, -1.2067, -0.3588],
         [ 0.6795,  0.8456, -1.1249, -0.4002],
         [ 0.6304,  0.8405, -1.0339, -0.4369],
         ...,
         [ 0.4987,  0.8106, -0.8202, -0.4891],
         [ 0.2261,  0.7062, -0.4481, -0.4842],
         [ 0.6606,  0.8442, -1.0889, -0.4159]],

        [[ 0.7269,  0.8607, -1.2420, -0.3456],
         [ 0.6780,  0.8645, -1.1409, -0.4016],
         [ 0.6682,  0.8644, -1.1218, -0.4108],
         ...,
         [ 0.5312,  0.8461, -0.8862, -0.4910],
         [ 0.4273,  0.8181, -0.7313, -0.5140],
         [ 0.6581,  0.8641, -1.1027, -0.4195]]])

### Forward Pass: Student
Student encodes the Context (Control), With the student we need to accumulate gradients since we need to be able to train the model. 
> TO DO: Add masking here if you want extra difficulty, but the task Control->Treated is enough for now.

In [17]:
context_latents = student(x_control)
context_latents

tensor([[[ 0.7095,  0.8514, -1.1911, -0.3698],
         [ 0.6285,  0.8470, -1.0360, -0.4395],
         [ 0.6349,  0.8478, -1.0474, -0.4353],
         ...,
         [ 0.5150,  0.8236, -0.8499, -0.4888],
         [ 0.3613,  0.7735, -0.6292, -0.5055],
         [ 0.6306,  0.8473, -1.0398, -0.4381]],

        [[ 0.7263,  0.8471, -1.2229, -0.3504],
         [ 0.6243,  0.8420, -1.0250, -0.4412],
         [ 0.6869,  0.8480, -1.1414, -0.3935],
         ...,
         [ 0.5421,  0.8252, -0.8886, -0.4787],
         [ 0.4739,  0.8062, -0.7850, -0.4951],
         [ 0.6548,  0.8458, -1.0801, -0.4205]]],
       grad_fn=<NativeLayerNormBackward0>)

## Predictor Setup
We need to create a predictor that tries to guess Target (from the teacher) given Context + Action


**Adaptive Layer Normalization AdaLN**

In [18]:
class AdaLN(nn.Module):
    '''
    Adaptive Layer Norm for conditioning the predictor on action embeddings.
    The action vector regresses the Scale (gamma) and Shift (beta) of the normalization.
    '''
    def __init__(self, embed_dim, action_embed_dim):
        super().__init__()
        self.norm = nn.LayerNorm(embed_dim, elementwise_affine=False)
        self.action_mlp = nn.Sequential(
            nn.SiLU(),
            nn.Linear(action_embed_dim, 2 * embed_dim)
        )
        # Initialize to identity (gamma=0, beta=0 originally, effectively gamma=1 after logic)
        # Zero-init the last layer so the action starts as a "no-op" (identity)
        nn.init.zeros_(self.action_mlp[1].weight)
        nn.init.zeros_(self.action_mlp[1].bias)

    def forward(self, x, action_emb):
        # x: [Batch, Seq, Dim]
        # action_emb: [Batch, action_embed_dim]
        
        # Project action to style: [B, 2*D] -> [B, 1, 2*D]
        style = self.action_mlp(action_emb).unsqueeze(1) 
        gamma, beta = style.chunk(2, dim=-1)
        
        # Apply affine transformation based on action
        return self.norm(x) * (1 + gamma) + beta

**Predictor Block**

In [19]:
class PredictorBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        # 1. Conditioning (AdaLN) replaces standard LayerNorm
        self.ada_ln1 = AdaLN(config.embed_dim, config.action_embed_dim)
        
        # 2. Attention (Using the shared BioMultiHeadAttention)
        self.attn = BioMultiHeadAttention(config)
        
        # 3. Conditioning (AdaLN) for the MLP block
        self.ada_ln2 = AdaLN(config.embed_dim, config.action_embed_dim)
        
        # 4. MLP (Using the shared MLP)
        self.mlp = MLP(config)

    def forward(self, x, action_emb, cos, sin):
        # 1. AdaLN -> Attention (with internal RoPE) -> Residual
        x_norm = self.ada_ln1(x, action_emb)
        
        # Note: BioMultiHeadAttention handles q/k/v projection and apply_rotary_pos_emb internally
        attn_out = self.attn(x_norm, cos, sin)
        x = x + attn_out
        
        # 2. AdaLN -> MLP -> Residual
        x_norm = self.ada_ln2(x, action_emb)
        x = x + self.mlp(x_norm)
        
        return x

**Main Predictor Model**

In [20]:
@dataclass
class ACPredictorConfig:
    num_pathways: int = 1024
    n_layer: int = 6 
    heads: int = 4
    embed_dim: int = 384
    action_embed_dim: int=256 
    mlp_ratio: float = 4.0
    max_perturb: int= 2058 ## eventually try to get to a 2**N power

class ACPredictor(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # Action Embedding (Discrete ID -> Vector)
        self.action_embed = nn.Embedding(config.max_perturb, config.action_embed_dim)
        
        # Learnable Queries ("Mask Tokens") for the future state
        # One query vector per pathway position
        self.mask_queries = nn.Parameter(torch.randn(1, config.num_pathways, config.embed_dim) * 0.02)
        
        # RoPE: initialized with HEAD dimension (dim // heads)
        head_dim = config.embed_dim // config.heads
        self.rope = RotaryEmbedding(head_dim)
        
        self.blocks = nn.ModuleList([
            PredictorBlock(config) for _ in range(config.n_layer)
        ])
        
        self.final_norm = AdaLN(config.embed_dim, config.action_embed_dim)
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, context_latents, action_ids):
        """
        context_latents: [Batch, N, Dim] (From Student Encoder)
        action_ids: [Batch] (Ints)
        """
        B, N, D = context_latents.shape
        
        # 1. Embed Action
        action_emb = self.action_embed(action_ids) # [B, action_embed_dim]
        
        # 2. Construct Input: [Context, Mask_Queries]
        # We concatenate the learned queries to the context. 
        # The predictor will attend to the context to update the queries.
        queries = self.mask_queries.repeat(B, 1, 1) # [B, N, D]
        sequence = torch.cat([context_latents, queries], dim=1) # [B, 2N, D]
        
        # 3. Generate RoPE for the full sequence (2N)
        # cos, sin are [2N, Head_Dim]
        cos, sin = self.rope(sequence)
        cos, sin = cos.to(sequence.device), sin.to(sequence.device)
        
        # 4. Pass through AdaLN Blocks
        for block in self.blocks:
            sequence = block(sequence, action_emb, cos, sin)
            
        sequence = self.final_norm(sequence, action_emb)
        
        # 5. Return only the predicted part (The Queries corresponding to N..2N)
        predictions = sequence[:, N:, :] 
        return predictions

### Create the Predictor Model

In [21]:
predictor = ACPredictor(ACPredictorConfig(n_layer=1, heads=n_heads, embed_dim=n_embd, action_embed_dim=n_embd))
predictor

ACPredictor(
  (action_embed): Embedding(2058, 4)
  (rope): RotaryEmbedding()
  (blocks): ModuleList(
    (0): PredictorBlock(
      (ada_ln1): AdaLN(
        (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=False)
        (action_mlp): Sequential(
          (0): SiLU()
          (1): Linear(in_features=4, out_features=8, bias=True)
        )
      )
      (attn): BioMultiHeadAttention(
        (q_proj): Linear(in_features=4, out_features=4, bias=True)
        (k_proj): Linear(in_features=4, out_features=4, bias=True)
        (v_proj): Linear(in_features=4, out_features=4, bias=True)
        (c_proj): Linear(in_features=4, out_features=4, bias=True)
      )
      (ada_ln2): AdaLN(
        (norm): LayerNorm((4,), eps=1e-05, elementwise_affine=False)
        (action_mlp): Sequential(
          (0): SiLU()
          (1): Linear(in_features=4, out_features=8, bias=True)
        )
      )
      (mlp): MLP(
        (c_fc): Linear(in_features=4, out_features=16, bias=True)
        (gelu)

### Forward Pass: Action Predictor
The Action predictor tries to guess Target encoding given Context + Action.  The context is the ouput of the Student and the action is from our tokenized data.  The goal here is tha the output attempts to match the teacher. 

In [22]:
action_id

tensor([1499,  777])

In [23]:
predicted_latents = predictor(context_latents, action_id)
predicted_latents

tensor([[[-1.3121, -0.4020,  0.3994,  1.3180],
         [ 1.5843, -0.7185, -0.8059, -0.0594],
         [-0.3733,  1.6866, -0.8537, -0.4579],
         ...,
         [ 0.4309,  1.2771, -1.1615, -0.5455],
         [-1.5878, -0.0927,  0.7758,  0.9083],
         [ 1.3567, -0.6971, -1.1521,  0.4932]],

        [[-1.3147, -0.4032,  0.4006,  1.3160],
         [ 1.5850, -0.7198, -0.8036, -0.0621],
         [-0.3746,  1.6880, -0.8511, -0.4605],
         ...,
         [ 0.4306,  1.2780, -1.1584, -0.5485],
         [-1.5906, -0.0935,  0.7768,  0.9061],
         [ 1.3571, -0.6986, -1.1494,  0.4908]]], grad_fn=<SliceBackward0>)

# Loss Calculation

We'll use L1 Loss (Mean Absolute Error) which measures the average absolute difference between predicted and actual values, making it robust to outliers.  We'll call this latent loss since we'll  use an L1 loss function within the latent space of a model to ensure the compressed representation (latent) accurately reflects the original data. 

In [24]:
loss = F.l1_loss(predicted_latents, target_latents)
loss

tensor(1.0230, grad_fn=<MeanBackward0>)

# Back Propogation

In [25]:
loss.backward()

## Gradient Clipping

In [26]:
torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)
torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)

tensor(5.8341)

## Update the Teacher

Recall that our teacher was created from a copy of our student. This means the weights are initiated randomly. We also didn't include any backprop for our teacher as since our loss is calculated using the action predictor and the student model.  To update the teacher we'll use a mechanism called  **Exponential Moving Average (EMA)**

In simple terms, this mechanism forces the Teacher to evolve very slowly, becoming a stable, wiser version of the Student.

Here is the breakdown of the mechanics. Mathematically, it calculates:

$Teacher_{new} = (m * Teacher_{old})+((1−m)*Student_{current})$

With a typical momentum ($m$) of 0.996, the code tells the Teacher: *"Keep 99.6% of your current weights, and mix in just 0.4% of what the Student learned in this last step."*

The reason we do this is Stability. The Student network is updating rapidly and noisily using Gradient Descent ($Param←Param−LearningRate×Gradient$). If we simply copied the Student to the Teacher (`Teacher = Student`), the target we are trying to predict would move erratically every single step, making it impossible to learn (like trying to hit a bullseye that is vibrating wildly).

By updating the Teacher this way, it effectively represents the **average** of the Student's weights over the last several thousand steps. This smooths out the noise and provides a steady, high-quality target for the Student to predict, preventing "Model Collapse" (where the model cheats by outputting zeros for everything).

TO:DO  we could also replace the teacher with a pretrained model and use this as an update mechanism. 

In [28]:
m=0.996
for param_s, param_t in zip(student.parameters(), teacher.parameters()):
    param_t.data.mul_(m).add_((1 - m) * param_s.data)

## Summary
Now we've done a walkthrough of the forward pass of the model.  With this we can see a lot of room for improvement but a decent starting point. 